In [165]:
import pandas as pd
from sklearn.neighbors import KernelDensity
import numpy as np
import matplotlib.pyplot as plt

In [166]:
def scores_min_max_temp(T_min_modele,T_max_modele,sensibilite,Temp_Abs_Min,Temp_Abs_Max):
    Score_min_max = 1
    if T_min_modele <= Temp_Abs_Min:
        a = ((1 - 0.5)/(Temp_Abs_Min - (Temp_Abs_Min - sensibilite)))
        b = 1 - a*Temp_Abs_Min
        Score_min_max = a*T_min_modele + b
    if T_max_modele >= Temp_Abs_Max:
        a = -(1 - 0.5)/(Temp_Abs_Max - (Temp_Abs_Max - sensibilite))
        b = 1- a*Temp_Abs_Max
        Score_min_max = a*T_max_modele + b
    if Score_min_max < 0:
        Score_min_max = 0
    return Score_min_max


def scores_temp(T_moy_modele,T_min_modele,T_max_modele,temp_opt_min,Temp_Opt_Max,Temp_Abs_Min,Temp_Abs_Max, sensibilite):
    Score_min_max = scores_min_max_temp(T_min_modele,T_max_modele,sensibilite,Temp_Abs_Min,Temp_Abs_Max)
    Score = 1
    if T_moy_modele <= temp_opt_min:
        a = ( 1 - 0 )/(temp_opt_min - Temp_Abs_Min) 
        b = 1 - a*temp_opt_min
        Score = a*T_moy_modele + b
    if T_moy_modele >= Temp_Opt_Max:
        a = (1 - 0 )/(Temp_Opt_Max - Temp_Abs_Max)
        b = 1 - a*Temp_Opt_Max
        Score = a*T_moy_modele + b
    if Score < 0:
        Score = 0
    Score_tot = Score_min_max*Score
    return Score_tot,Score_min_max,Score

def Score_Moyglissante_temp(Score_Mois,temps_pousse):
    Moy_glissante = []
    for i in range(12):
        Score_Mois.append(Score_Mois[i])
        Moy_calcul = Score_Mois[0+i:i+temps_pousse]
        Score = 1
        for donnee in Moy_calcul:
            #print (donnee)
            Score = Score*donnee
        Moy_glissante.append(Score)
    Score_max = np.max(Moy_glissante)
    periode = np.argmax(Moy_glissante)
    return Score_max,periode

def scores_pluv(Rain_modele,Rain_Opt_min,Rain_Opt_Max,Rain_Abs_Min,Rain_Abs_Max):
    """Rain_modele : le cumul de pluviométrie prevue 
    Rain_opt_min : born inferieur de la pluviométrie optimal 
    Rain_Opt_Max : born superieur de la pluviométrie optimal
    Rain_abs_min : pluviometrie minimal accepter par la plante
    Rain_abs_max : pluviometrie maximal accepter par la plante"""
    Score = 1
    if Rain_modele <= Rain_Opt_min:
        a = ( 1 - 0 )/(Rain_Opt_min - Rain_Abs_Min) 
        b = 1 - a*Rain_Opt_min
        Score = a*Rain_modele + b
    if Rain_modele >= Rain_Opt_Max:
        a = (1 - 0 )/(Rain_Opt_Max - Rain_Abs_Max)
        b = 1 - a*Rain_Opt_Max
        Score = a*Rain_modele + b
    if Score < 0:
        Score = 0
    return Score

In [167]:
### LOAD DATA ####
climate_df = pd.read_csv('indices_France_2020-2050-mensuelles.txt',sep=";")
display (climate_df.head(30))
climate_df = climate_df.iloc[:,0:-1]
#print(climate_df.iloc[:,5:].describe())
#print(climate_df.describe())
#print(climate_df.columns)
climate_df.columns = ["Point",'Latitude', 'Longitude', 'Contexte', 'Période', 'Mois','Tmoy','Tmin','Tmax','NORTRAV', 'NORTXQ90', 'NORTXQ10',
       'NORTNQ10', 'NORTNQ90', 'NORHDD', 'NORCDD', 'NORPAV', 'NORRR','NORPFL90', 'NORPINT']
print (climate_df.columns)
### TEST UN POINT UNE ESPECE ###
#Points = climate_df.unique(df['Point'])
#Point = Points[0]
#crop_une_station = climate_df[climate_df['Point']==282]
#print(crop_une_station)
#for indice in range(crop_une_station.shape[0]):
#    ligne = crop_une_station.iloc[indice,:]
#    print(ligne)
#    Score_point_mois_ = []

,Point,Latitude,Longitude,Contexte,Période,Mois,NORTAV,NORTNAV,NORTXAV,NORTRAV,...,NORTXQ10,NORTNQ10,NORTNQ90,NORHDD,NORCDD,NORPAV,NORRR,NORPFL90,NORPINT,Unnamed: 20
0,282,41.4087,9.1577,RCP2.6,H1,1,9.58,6.16,12.99,6.83,...,9.74,1.13,10.51,230.12,0.00,2.93,90.94,75.42,8.96,NaN
1,282,41.4087,9.1577,RCP2.6,H1,2,9.88,6.40,13.36,6.96,...,10.27,1.91,10.61,201.24,0.04,3.12,87.89,71.46,9.08,NaN
2,282,41.4087,9.1577,RCP2.6,H1,3,11.56,7.70,15.43,7.74,...,11.17,2.12,11.97,169.93,0.60,2.54,78.59,65.04,7.85,NaN
3,282,41.4087,9.1577,RCP2.6,H1,4,13.96,10.20,17.72,7.51,...,14.22,6.23,13.20,95.15,1.58,2.45,73.62,69.98,7.78,NaN
4,282,41.4087,9.1577,RCP2.6,H1,5,18.63,14.56,22.69,8.13,...,17.46,10.69,18.05,19.31,51.10,1.57,48.70,57.13,6.62,NaN
5,282,41.4087,9.1577,RCP2.6,H1,6,21.99,17.63,26.34,8.72,...,22.82,14.79,20.11,0.40,121.00,0.82,24.53,40.56,5.68,NaN
6,282,41.4087,9.1577,RCP2.6,H1,7,25.78,21.12,30.45,9.33,...,26.95,18.37,23.36,0.00,241.23,0.42,13.17,28.21,7.11,NaN
7,282,41.4087,9.1577,RCP2.6,H1,8,26.79,22.10,31.48,9.38,...,27.78,19.23,24.29,0.00,272.49,0.52,16.04,40.52,8.09,NaN
8,282,41.4087,9.1577,RCP2.6,H1,9,22.36,17.96,26.77,8.81,...,23.33,14.75,20.62,0.58,132.13,2.60,78.12,78.72,9.72,NaN
9,282,41.4087,9.1577,RCP2.6,H1,10,17.85,13.75,21.94,8.18,...,18.53,9.80,17.18,19.88,28.87,2.72,84.18,78.83,9.18,NaN


Index(['Point', 'Latitude', 'Longitude', 'Contexte', 'Période', 'Mois', 'Tmoy',
       'Tmin', 'Tmax', 'NORTRAV', 'NORTXQ90', 'NORTXQ10', 'NORTNQ10',
       'NORTNQ90', 'NORHDD', 'NORCDD', 'NORPAV', 'NORRR', 'NORPFL90',
       'NORPINT'],
      dtype='object')


In [170]:
pheno_df = pd.read_csv('plantes_sol.csv',sep=",")
to_keep = ['species', 'temp_opt_min','Temp_Opt_Max', 'Temp_Abs_Min', 'Temp_Abs_Max', 'Rain_Opt_Min',
           'Rain_Opt_Max', 'Rain_Abs_Min', 'Rain_Abs_Max', 'pH_Opt_Min', 'pH_Opt_Max','pH_Abs_Min', 'pH_Abs_Max', 
           'temps_pousse']
pheno_df = pheno_df[to_keep]
display(pheno_df)

,species,temp_opt_min,Temp_Opt_Max,Temp_Abs_Min,Temp_Abs_Max,Rain_Opt_Min,Rain_Opt_Max,Rain_Abs_Min,Rain_Abs_Max,pH_Opt_Min,pH_Opt_Max,pH_Abs_Min,pH_Abs_Max,temps_pousse
0,Colza,15,25,5,41,500,1000,400,2800,6.5,7.6,5.5,8.0,12
1,Soja,20,33,10,38,600,1500,450,1800,5.5,6.5,4.5,8.4,5
2,Orge,15,20,2,40,500,1000,200,2000,6.5,7.5,6.0,8.0,8
3,Manioc,20,29,10,35,1000,1500,500,5000,5.5,8.0,4.0,9.0,7
4,Basilic,18,27,7,36,1000,1600,600,4300,6.0,7.0,4.3,8.2,6
5,Pomme de terre,15,25,7,30,500,800,250,2000,5.0,6.2,4.2,8.5,6
6,Cacao,21,32,10,38,1200,3000,900,7600,5.0,6.5,4.0,8.0,12
7,Mais,18,33,10,47,600,1200,400,1800,5.0,7.0,4.5,8.5,7
8,Lavande,17,26,10,30,600,1000,400,1200,6.5,7.0,5.8,7.5,6
9,Riz,25,35,16,38,1500,2000,1000,4000,5.0,8.0,3.5,9.0,6


In [171]:
def get_climate_metrics(climate_df, horizon, scenario):
    climate_metrics = climate_df[(climate_df['Période'] == horizon) & (climate_df['Contexte'] == scenario)]
    return climate_metrics

In [32]:
new = get_climate_metrics(climate_df, 'H1', 'RCP2.6')
print (type(new))
#print (new.info())
#print (new.head(30))
unique_points = new['Point'].unique()
#listofdfs = [new.loc[new['Point']==val, :] for val in new['Point'].unique()]
#print (listofdfs)
#list_df = 
list_df = new.grouby(ne)

<class 'pandas.core.frame.DataFrame'>


In [172]:
SENSIBILITE = 1

In [230]:
## 7000 points -> 7000 lignes 
## long, lat, score_totale, score_pluie, score_temp
def get_all_scores(climate_df, pheno_df, species, horizon, scenario):
    total_temp_score = []
    total_pluv_score = []
    total_lat = []
    total_long = []
    name_lst = []
    climate_metrics = get_climate_metrics(climate_df, horizon, scenario)
    species_metrics = pheno_df[pheno_df['species'] == species]
    species_metrics.reset_index(drop=True, inplace=True)
    temp_opt_min = species_metrics['temp_opt_min'][0]
    Temp_Opt_Max = species_metrics['Temp_Opt_Max'][0]
    Temp_Abs_Min = species_metrics['Temp_Abs_Min'][0]
    Temp_Abs_Max = species_metrics['Temp_Abs_Max'][0]
    Rain_Opt_min = species_metrics['Rain_Opt_Min'][0]
    Rain_Opt_Max = species_metrics['Rain_Opt_Max'][0]
    Rain_Abs_Min = species_metrics['Rain_Abs_Min'][0]
    Rain_Abs_Max = species_metrics['Rain_Abs_Max'][0]
    temps_pousse = species_metrics['temps_pousse'][0]
    unique = climate_metrics.groupby('Point')
    for name, group in unique:
        month_score = []
        #print (f'--NEW : {name}--')
        long = group['Longitude'].values[0]
        lat = group['Latitude'].values[0]
        for idx, row in group.iterrows():
            T_moy = row['Tmoy']
            T_min = row['Tmin']
            T_max = row['Tmax']
            score = scores(T_moy, T_min, T_max, temp_opt_min, Temp_Opt_Max, Temp_Abs_Min, Temp_Abs_Max, SENSIBILITE)
            month_score.append(score[0])
        total_score = Score_Moyglissante_temp(month_score, temps_pousse)
        period = total_score[1]
        cumul = group['NORRR']
        Rain_model = cumul[period:period+temps_pousse].sum()
        score_pluv = scores_pluv(Rain_model,Rain_Opt_min,Rain_Opt_Max,Rain_Abs_Min,Rain_Abs_Max)
        #print (total_score[0])
        #print (score_pluv)
        total_temp_score.append(total_score[0])
        total_pluv_score.append(score_pluv)
        total_lat.append(lat)
        total_long.append(long)
        name_lst.append(name)
    result_dict = {'Point': name_lst, 'Lat':total_lat, 'Long':total_long,
                   'Temp_score':total_temp_score, 'Pluv_score':total_pluv_score}
    result_df = pd.DataFrame(result_dict)
    display(result_df.sample(25))
    return result_df

In [231]:
new = get_all_scores(climate_df, pheno_df, 'Colza', 'H3', 'RCP2.6')

,Point,Lat,Long,Temp_score,Pluv_score
0,282,41.4087,9.1577,5.244122e-02,1.0
1,423,41.4925,8.9759,5.722815e-02,1.0
2,424,41.4864,9.0710,6.311247e-02,1.0
3,425,41.4802,9.1660,3.811304e-02,0.0
4,426,41.4739,9.2610,6.031276e-02,1.0
...,...,...,...,...,...
8597,18800,50.9780,2.2797,4.761655e-07,1.0
8598,18801,50.9780,2.3933,5.690877e-07,0.0
8599,18802,50.9779,2.5069,4.273868e-07,1.0
8600,18944,51.0497,2.3934,5.388320e-07,1.0


In [232]:
display(new.sample(20))

,Point,Lat,Long,Temp_score,Pluv_score
3575,8799,45.9418,3.2135,0.000000e+00,1.000000
5990,12879,47.8645,-3.9247,1.480960e-03,1.000000
626,3652,43.3512,3.2725,1.293185e-02,1.000000
4039,9655,46.3751,3.0125,0.000000e+00,1.000000
1486,5104,44.0291,5.4776,0.000000e+00,0.951333
6729,13802,48.4617,3.0394,0.000000e+00,1.000000
2907,7624,45.3458,0.0399,0.000000e+00,1.000000
6756,13829,48.4084,5.9567,0.000000e+00,1.000000
3177,8080,45.5844,2.7978,0.000000e+00,0.831500
1029,4371,43.7065,3.6748,0.000000e+00,1.000000


In [ ]:
# RAIN model -> sum ( de period a temps_pousse + period)
# 